In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer  # 피체 벡터화
from sklearn.metrics.pairwise import cosine_similarity  # 코사인 유사도

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time

In [2]:
'''
데이터 파일 로드
'''
Gyeonggi = pd.read_csv('소상공인시장진흥공단_상가(상권)정보_경기_202206.CSV',encoding='utf-8')
Seoul = pd.read_csv('소상공인시장진흥공단_상가(상권)정보_서울_202206.csv',encoding='utf-8')
incheon = pd.read_csv('소상공인시장진흥공단_상가(상권)정보_인천_202206.csv',encoding='utf-8')

In [3]:
'''
서울, 경기, 인천 음식점
1. 음식점만 남겨두고 제거
2. 필요한 열을 제외하고 삭제
3. 열의 순서를 다시부여
'''

Gyeonggi_food = Gyeonggi.copy()
Seoul_food = Seoul.copy()
incheon_food = incheon.copy()
################################################################################################################
# 경기도 
Gyeonggi_food.drop(Gyeonggi_food[Gyeonggi_food['상권업종대분류명'] != "음식"].index, inplace=True)
Gyeonggi_food.drop(Gyeonggi_food[Gyeonggi_food['상권업종중분류명'] == '제과제빵떡케익'].index, inplace=True)
Gyeonggi_food.drop(Gyeonggi_food[Gyeonggi_food['상권업종중분류명'] == '음식배달서비스'].index, inplace=True)
Gyeonggi_food.drop(Gyeonggi_food[Gyeonggi_food['상권업종중분류명'] == '유흥주점'].index, inplace=True)
Gyeonggi_food.drop(Gyeonggi_food[Gyeonggi_food['상권업종중분류명'] == '커피점/카페'].index, inplace=True)

Gyeonggi_food = Gyeonggi_food[['상호명', '상권업종중분류명',
                                '상권업종소분류명','도로명','경도', '위도']]

Gyeonggi_food.reset_index(inplace=True)
Gyeonggi_food.drop(['index'],axis=1,inplace=True)
################################################################################################################
#서울
Seoul_food.drop(Seoul_food[Seoul_food['상권업종대분류명'] != "음식"].index, inplace=True)
Seoul_food.drop(Seoul_food[Seoul_food['상권업종중분류명'] == '제과제빵떡케익'].index, inplace=True)
Seoul_food.drop(Seoul_food[Seoul_food['상권업종중분류명'] == '음식배달서비스'].index, inplace=True)
Seoul_food.drop(Seoul_food[Seoul_food['상권업종중분류명'] == '유흥주점'].index, inplace=True)
Seoul_food.drop(Seoul_food[Seoul_food['상권업종중분류명'] == '커피점/카페'].index, inplace=True)

Seoul_food = Seoul_food[['상호명', '상권업종중분류명',
                                '상권업종소분류명','도로명','경도', '위도']]

Seoul_food.reset_index(inplace=True)
Seoul_food.drop(['index'],axis=1,inplace=True)
################################################################################################################
#인천
incheon_food.drop(incheon_food[incheon_food['상권업종대분류명'] != "음식"].index, inplace=True)
incheon_food.drop(incheon_food[incheon_food['상권업종중분류명'] == '제과제빵떡케익'].index, inplace=True)
incheon_food.drop(incheon_food[incheon_food['상권업종중분류명'] == '음식배달서비스'].index, inplace=True)
incheon_food.drop(incheon_food[incheon_food['상권업종중분류명'] == '유흥주점'].index, inplace=True)
incheon_food.drop(incheon_food[incheon_food['상권업종중분류명'] == '커피점/카페'].index, inplace=True)

incheon_food = incheon_food[['상호명', '상권업종중분류명',
                                '상권업종소분류명','도로명','경도', '위도']]

incheon_food.reset_index(inplace=True)
incheon_food.drop(['index'],axis=1,inplace=True)
################################################################################################################

In [4]:
Gyeonggi_food

,상호명,상권업종중분류명,상권업종소분류명,도로명,경도,위도
0,큰손의정부부대찌개,한식,부대찌개/섞어찌개,경기도 김포시 감정로,126.699974,37.620590
1,앞족애,한식,한식/백반/한정식,경기도 의정부시 둔야로,127.039861,37.739330
2,스푼,한식,한식/백반/한정식,경기도 파주시 정담길,126.773696,37.770514
3,동해안,한식,한식/백반/한정식,경기도 파주시 마무리길,126.778020,37.760224
4,독도해물알찜,한식,한식/백반/한정식,경기도 평택시 비전5로,127.114789,37.000418
...,...,...,...,...,...,...
126374,스시마아토,일식/수산물,음식점-초밥전문,경기도 가평군 가평읍 장터2길,127.515182,37.829716
126375,인간화로구이,중식,중국음식/중국집,경기도 안산시 단원구 원본로,126.794185,37.329987
126376,김종구부산어묵,분식,라면김밥분식,경기도 고양시 일산동구 숲속마을1로,126.799709,37.667310
126377,김서영완도맛집,한식,한식/백반/한정식,경기도 파주시 탄현면 검산로,126.738417,37.804703


In [9]:
# Gyeongi_food2 = Gyeonggi_food
# Gyeongi_food2['naver_keyword2'] = ""

In [8]:
# Gyeongi_food2['naver_keyword2'] = Gyeonggi_food['naver_keyword'].str.replace(" ","%20")

In [10]:
# Gyeonggi_food2.head()

In [11]:
# print('https://map.naver.com/v5/search/'+Gyeonggi_food2['naver_keyword2'])

In [12]:
# list=[]
# for i in range(len(Gyeonggi_food2)):
#     link='https://map.naver.com/v5/search/'
#     link+=Gyeonggi_food2['naver_keyword2'][i]
#     list.append(link)

In [13]:
# list

In [14]:
# Gyeonggi_food2['naver_map_url'] = list

In [15]:
# Gyeonggi_food2

In [16]:
Gyeonggi_food['naver_keyword'] = Gyeonggi_food['상호명'] + "%20" + Gyeonggi_food['도로명']  # "%20"는 띄어쓰기를 의미합니다.
Gyeonggi_food['naver_map_url'] = ''
Gyeonggi_food['store_open'] = ''

In [17]:
chromedriver = '\\Program Files\\Google\\Chrome\\Application\\chrome.exe' # 셀레늄이 이용할 크롤링 드라이버 디렉토리를 입력
driver = webdriver.Chrome(chromedriver)

C:\Users\TJ\AppData\Local\Temp\ipykernel_9108\990080391.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver)


In [18]:
for i, keyword in enumerate(Gyeonggi_food['naver_keyword'].tolist()) : 
    print("이번에 찾을 키워드 :", i, f"/ {Gyeonggi_food.shape[0] -1} 행", keyword)
    try:
#         naver_map_search_url = f"https://m.map.naver.com/search2/search.naver?query={keyword}&sm=hty&style=v5"
        naver_map_search_url = f"https://map.naver.com/v5/search/{keyword}"
        driver.get(naver_map_search_url)
        
        # keyword로 검색해서 만약 업체가 없다면?
#         if == '조건에 맞는 업체가 없습니다.' :
#             Gyeonggi_food['store_open'] = 'X'
#         else :
#             Gyeonggi_food['store_open'] = 'O'
        
        time.sleep(5)
        Gyeonggi_food.iloc[i,-1] = driver.find_element_by_class_name("Gyeonggi_food")
        # 네이버 지도 시스템은 data-cid에 url 파라미터를 저장해두고 있었습니다.
        # data-cid 번호를 뽑아두었다가 기본 url 템플릿에 넣어 최종적인 url을 완성하면 됩니다.
        
        #만약 검색 결과가 없다면?
    except Exception as e1:
        if "li:nth-child(1)" in str(e1):  # -> "child(1)이 없던데요?"
            try:
                df.iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
                time.sleep(1)
            except Exception as e2:
                print(e2)
                df.iloc[i,-1] = np.nan
                time.sleep(1)
        else:
            pass


driver.quit()

이번에 찾을 키워드 : 0 / 126378 행 큰손의정부부대찌개%20경기도 김포시 감정로


KeyboardInterrupt: 

In [19]:
Gyeonggi_food['map_star_review'] = ""
Gyeonggi_food['image'] = ""

In [99]:
# for i, keyword in enumerate(Gyeonggi_food['naver_keyword'].tolist()) : 
#     print("이번에 찾을 키워드 :", i, f"/ {Gyeonggi_food.shape[0] -1} 행",keyword)
# #     print(url)
# #     driver.get(url)
# #     time.sleep(20)
    
#     try:
#              naver_map_url = f"https://map.naver.com/v5/search/{keyword}"
            
#              naver_driver.get(naver_map_url)
#              naver_driver.implicitly_wait(15)

#              naver_driver.switch_to.frame('entryIframe')
#              naver_driver.implicitly_wait(10)

#              time.sleep(5)

#              star = naver_driver.find_element(By.CLASS_NAME,'PXMot.LXIwF').text
#              print(star)

# #     try:
# #         # 별점불러오기
# #         map_star_review = driver.find_element_by_css_selector("#ct > div.dAsGb > span.place_blind > em").text
# # #         map_star_review_list.append(map_star_review)
    
#     except Exception as e1:
#         pass
        
    
    
# driver.quit()

In [34]:
chromedriver = '\\Program Files\\Google\\Chrome\\Application\\chrome.exe' # 셀레늄이 이용할 크롤링 드라이버 디렉토리를 입력
driver = webdriver.Chrome(chromedriver)

C:\Users\TJ\AppData\Local\Temp\ipykernel_9108\990080391.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver)


In [33]:
map_star_review_list = []
Gyeonggi_food_image_list = []
no_food_store_list = []

no_star = "X"
no_image = "X"

In [35]:
for i, keyword in enumerate(Gyeonggi_food['naver_keyword'].tolist()):
    #print("이번에 찾을 키워드 :", i, f"/ {Gyeonggi_food.shape[0] -1} 행", keyword)
    
    try:
#         naver_map_url = f"https://map.naver.com/v5/search/{parse.quote(keyword)}"
        naver_map_url = f"https://map.naver.com/v5/search/{keyword}"
#         naver_mqp_url = 'https://map.naver.com/v5/search/%EC%A0%9C%EC%A3%BC%EB%8F%84%20%EC%9D%B8%ED%84%B0%ED%8C%8C%ED%81%AC%EB%A0%8C%ED%8A%B8%EC%B9%B4/place/1058167611?c=14082020.2873345,3962314.0822643,15,0,0,0,dh&isCorrectAnswer=true'
       
#         https://map.naver.com/v5/search/%EC%A0%9C%EC%A3%BC%EB%8F%84%20%EC%9D%B8%ED%84%B0%ED%8C%8C%ED%81%AC%EB%A0%8C%ED%8A%B8%EC%B9%B4/place/1058167611?c=14082020.2873345,3962314.0822643,15,0,0,0,dh&isCorrectAnswer=true
        
        driver.get(naver_map_url)
        driver.implicitly_wait(5)
        
        driver.switch_to.frame('entryIframe')
        driver.implicitly_wait(5)
        
        time.sleep(5)
        
        star = driver.find_element(By.CLASS_NAME,'PXMot.LXIwF').text
        star_point= star[star.find('\n')+1:star.find('/')]
        map_star_review_list.append(star_point)
        print(star_point)
        #print(map_star_review_list)
        
        #image = driver.find_element(By.CLASS_NAME,'K0PDV _div')[1].text
        #Gyeonggi_food_image_list.append(image)
        
#         naver_html = urllib.request.urlopen(naver_map_url)
#         naver_soup = BeautifulSoup(naver_html, 'html.parser')
#         naver_driver.implicitly_wait(3)
        
#        time.sleep(300)
        
#         print(review)
        #print('\n'*3)
            
        
    except Exception as e1:
        map_star_review_list.append(no_star)
        #Gyeonggi_food_image_list.append(no_image)

    
    
driver.quit()

# naver_df['naver_map_url'] = "https://map.naver.com/v5/search/"+naver_df['naver_keyword']

# naver_df = naver_df.loc[~naver_df['naver_map_url'].isnull()]

KeyboardInterrupt: 

In [29]:
#Gyeonggi_food['map_star_review'] = map_star_review_list